In [3]:
from mzml2isa.mzml import MzMLFile
import json
import openpyxl
from pathlib import Path
import urllib3
import pandas as pd


http = urllib3.PoolManager()
url = "https://github.com/nfdi4plants/SWATE_templates/raw/main/templates/3ASY02_ProteomicsMassSpec/3ASY02_ProteomicsMassSpec.xlsx"
xlsx = http.request('GET', url)


swate_template = pd.read_excel(url,'3ASY02_ProteomicsMassSpec')


mass_spec_list =["Parameter [Experiment type]",
"Parameter [Quantification method]",
"Parameter [technical replicate]",
"Parameter [Variable modification]",
"Parameter [Fixed modification]",
"Parameter [sample volume]",
"Parameter [injection volume]",
"Parameter [count unit]",
"Parameter [instrument model]",
"Parameter [duration]"]

In [6]:
sheet = swate_template

In [16]:
sheet.iloc[0,3:]

Unnamed: 3                           Parameter [Experiment type]
Unnamed: 4     Term Source REF [Experiment Type] (#h; #tPRIDE...
Unnamed: 5     Term Accession Number [Experiment Type] (#h; #...
Unnamed: 6                     Parameter [Quantification method]
Unnamed: 7          Term Source REF [Quantification method] (#h)
Unnamed: 8     Term Accession Number [Quantification method] ...
Unnamed: 9                       Parameter [technical replicate]
Unnamed: 10    Term Source REF [technical replicate] (#h; #tM...
Unnamed: 11    Term Accession Number [technical replicate] (#...
Unnamed: 12                    Parameter [Variable modification]
Unnamed: 13         Term Source REF [Variable modification] (#h)
Unnamed: 14    Term Accession Number [Variable modification] ...
Unnamed: 15                       Parameter [Fixed modification]
Unnamed: 16            Term Source REF [Fixed modification] (#h)
Unnamed: 17      Term Accession Number [Fixed modification] (#h)
Unnamed: 18              

In [144]:
ont = []
for row in sheet.iter_rows( min_col =3, min_row =2, max_row=2):
    for cell in row:
        if cell.value[0:15] =="Term Source REF":
            print(cell.value ) 
            try: 
                ontology = cell.value.split("#t")[1]
                print(ontology[:-1])
                ont.append(ontology[:-1])
            except IndexError :
                print("no_ontoloy")
                ont.append("")

Term Source REF [Experiment Type] (#h; #tPRIDE:0000457)
PRIDE:0000457
Term Source REF [Quantification method] (#h)
no_ontoloy
Term Source REF [technical replicate] (#h; #tMS:1001808)
MS:1001808
Term Source REF [Variable modification] (#h)
no_ontoloy
Term Source REF [Fixed modification] (#h)
no_ontoloy
Term Source REF [sample volume] (#h; #tMS:1000005)
MS:1000005
Term Source REF [injection volume] (#h)
no_ontoloy
Term Source REF [count unit] (#h; #tUO:0000189)
UO:0000189
Term Source REF [instrument model] (#h; #tMS:1000031)
MS:1000031
Term Source REF [duration] (#h; #tPATO:0001309)
PATO:0001309


In [145]:
ont

['PRIDE:0000457',
 '',
 'MS:1001808',
 '',
 '',
 'MS:1000005',
 '',
 'UO:0000189',
 'MS:1000031',
 'PATO:0001309']

In [108]:
help(sheet.iter_rows)

Help on method iter_rows in module openpyxl.worksheet.worksheet:

iter_rows(min_row=None, max_row=None, min_col=None, max_col=None, values_only=False) method of openpyxl.worksheet.worksheet.Worksheet instance
    Produces cells from the worksheet, by row. Specify the iteration range
    using indices of rows and columns.
    
    If no indices are specified the range starts at A1.
    
    If no cells are in the worksheet an empty tuple will be returned.
    
    :param min_col: smallest column index (1-based index)
    :type min_col: int
    
    :param min_row: smallest row index (1-based index)
    :type min_row: int
    
    :param max_col: largest column index (1-based index)
    :type max_col: int
    
    :param max_row: largest row index (1-based index)
    :type max_row: int
    
    :param values_only: whether only cell values should be returned
    :type values_only: bool
    
    :rtype: generator



In [146]:
mzml_name_list= [["Experiment type",
"Quantification method",
"technical replicate",
"Variable modification",
"Fixed modification",
"sample volume",
"injection volume",
"count unit",
"Instrument",
"duration"],
                ["Experiment type",
"Quantification method",
"technical replicate",
"Variable modification",
"Fixed modification",
"sample volume",
"injection volume",
"count unit",
"name",
"duration"],
                
                ont]



In [147]:
# pso = "NFDI4PSO:0000000"
full_list = []
for i,parameters in enumerate(mass_spec_list):
    full_list.append(dict( swate_name = parameters, nfdi4pso = mzml_name_list[2][i], mzml_key1 = mzml_name_list[0][i], mzml_key2 = mzml_name_list[1][i]  ))

In [73]:

mzml_metadata = MzMLFile('mzml/','4.mzml')



# mzml_metadata.metadata['Instrument serial number']['value']
# mzml_metadata.metadata['Instrument manufacturer']['name']
# mzml_metadata.metadata['Instrument software version']['value']
# mzml_metadata.metadata['Scan polarity']['name']


# mzml_metadata.metadata['Mass analyzer']['name']
# mzml_metadata.metadata['Instrument']['accession']
# # mzml_metadata.metadata['Instrument software']['name']

# # f.metadata['Scan polarity']['Instrument software version']


'http://purl.obolibrary.org/obo/MS_1002533'

In [148]:
def search_mzml_name(mzml_metadata , dict_name, dict_value, ):
    '''
    a function to search the mzml metadata by dictionary names and value, it also handels exceptions
    '''
    try:
        mzml_metadata.metadata[dict_name][dict_value]
        return [mzml_metadata.metadata[dict_name]['accession'], mzml_metadata.metadata[dict_name][dict_value]]
    except KeyError:
        return [ 'no relavent ontology', 'please fill this cell manually' ]
        

In [151]:
for i,dict_row in enumerate(full_list):
    
    ont, value = search_mzml_name(mzml_metadata, dict_row['mzml_key1'], dict_row['mzml_key2'] )
    dict_row.update(dict( mzml_output = value, mzml_ont = ont ))

with open('data.json', 'w') as f:
    json.dump(full_list, f)

In [56]:
search_mzml_name(mzml_metadata, 'Instrument manufacturer','name')
search_mzml_name(mzml_metadata, 'Instrument manufacturer','value')

['no relavent ontology', 'please fill this cell manually']

In [49]:
mzml_metadata.metadata

{'Data file content': {'entry_list': [{'accession': 'http://purl.obolibrary.org/obo/MS_1000579',
    'name': 'MS1 spectrum',
    'ref': 'MS'},
   {'accession': 'http://purl.obolibrary.org/obo/MS_1000580',
    'name': 'MSn spectrum',
    'ref': 'MS'},
   {'accession': 'http://purl.obolibrary.org/obo/MS_1000235',
    'name': 'total ion current chromatogram',
    'ref': 'MS'},
   {'accession': 'http://purl.obolibrary.org/obo/MS_1000628',
    'name': 'basepeak chromatogram',
    'ref': 'MS'}]},
 'Native spectrum identifier format': {'accession': 'http://purl.obolibrary.org/obo/MS_1000770',
  'name': 'WIFF nativeID format',
  'ref': 'MS'},
 'Raw data file format': {'accession': 'http://purl.obolibrary.org/obo/MS_1000562',
  'name': 'ABI WIFF format',
  'ref': 'MS'},
 'Data file checksum type': {'accession': 'http://purl.obolibrary.org/obo/MS_1000569',
  'name': 'SHA-1',
  'ref': 'MS',
  'value': '61c2723c84e53414804c5477af30d8e7b1cfa0ff'},
 'Ion source': {'accession': 'http://purl.obolibrar